Script to output the necessary fields for the VWIND constraint for CMIP6.  This includes...

Ensemble mean past and future v for all scales and zonal wavenumbers greater than 3 in map form.
Southwest meridional wind indices (250E-265E, 25N-37N) for past and future, ensemble mean and first member.
Past stationary wave amplitude for k>3 averaged over 200E-300E, 20N-40N for ensemble mean and first member.

Output is put to ecpaper2020/DATASORT/VWIND/DATA/

In [9]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import xesmf as xe
from numpy import nan
import sys
import math
import warnings
from glob import glob

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import filter_utils as filt
from ecpaper_utils import averaging_utils as avg

importlib.reload(read)
importlib.reload(cal)
importlib.reload(filt)
importlib.reload(avg)

warnings.filterwarnings('ignore')



Set paths for CMIP5 models (historical and RCP8.5) and the pressure level to us (in Pa)

In [10]:
histpath="/project/cmip6/historical/Amon/"
rcp85path="/project/cmip6/ssp585/Amon/"
plevuse="30000"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

Information on the models being used is provided in cmip5csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [11]:
cmip6models=pd.read_csv("../cmip6csvinfo.csv")

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

In [12]:
models=cmip6models["Model"]
nmodels=models.size

# define 1 deg greid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

# define multi-model arrays
vpast_em = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vpast_em')
vpast_1mem = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vpast_1mem')
vfut_em = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vfut_em')
vfut_1mem = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vfut_1mem')

# loop over models
for index, modname in models.iteritems():
    
    # initialize boolean for reusing weights
    wgtfile="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/tmp/wgt_cmip6.nc"
    reusewgt=False
    
    # ---- the past period ----
    nmems=cmip6models.loc[index, "Nmempast"]
    ftype=cmip6models.loc[index, "ftypep"]
    ptype=cmip6models.loc[index, "ptype"]
    
    for imem in range(1, nmems+1, 1): # loop over members
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        # check for a special order
        changeorder = cmip6models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr="r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr="r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
                
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir=glob(histpath+"va/"+modname+"/"+memstr+"/*/")
        histdir=histdir[0]
        
        
        # read in meridional wind, concatenate historical to 2005, rcp85 to 2014
        v = read.read_1lev(histdir+"/*.nc", datebegp, dateendp, plevuse) 

        # check the array length. 
        if (v.time.size != nmonthsp):
            print("something's wrong, nmonthsp="+str(nmonthsp)+" but v has size "+str(v.time.size))
            sys.exit("v past for "+modname+", "+memstr+" doesn't have the right size")
            
        # calculate DJF mean and interpolate
        vdjf = cal.season_mean(v, "va", season="DJF")
        regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
        vdjfinterp = regridder(vdjf)
        
        if (imem == 1):
            vpast_1mem[index,:,:] = vdjfinterp
            reusewgt=True
            
        vpast_em[index,:,:] = vpast_em[index,:,:] + vdjfinterp[:,:]/float(nmems)
        
    # ---the future period-------
    
    nmems=cmip6models.loc[index, "Nmemfuture"]
    ftype=cmip6models.loc[index, "ftypef"]
    for imem in range(1,nmems+1,1):
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        changeorder=cmip6models.loc[index,"specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr="r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr="r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
            
        print("Processing future for "+modname+" "+memstr)
        rcp85dir=glob(rcp85path+"/va/"+modname+"/"+memstr+"/*/")
        rcp85dir=rcp85dir[0]
        v=read.read_1lev(rcp85dir+"/*.nc", datebegf, dateendf, plevuse)
        
        # check the right number of months are there
        if (v.time.size != nmonthsf):
            print("something's wrong, nmonthf="+str(nmonthsf)+" but v has size "+str(v.time.size))
            sys.exit("v future for "+modname+", "+memstr+" doesn't have the right size")
            
        vdjf = cal.season_mean(v, "va", season="DJF")
        regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
        vdjfinterp = regridder(vdjf)
        
        if (imem == 1):
            vfut_1mem[index,:,:]=vdjfinterp
            reusewgt=True
            
        vfut_em[index,:,:] = vfut_em[index,:,:] + vdjfinterp[:,:]/float(nmems)
        
    # clean up regridding file
    !rm $wgtfile
    

Processing past for ACCESS-CM2 r1i1p1f1...
Processing past for ACCESS-CM2 r2i1p1f1...
Processing future for ACCESS-CM2 r1i1p1f1
Processing past for ACCESS-ESM1-5 r1i1p1f1...
Processing past for ACCESS-ESM1-5 r2i1p1f1...
Processing past for ACCESS-ESM1-5 r3i1p1f1...
Processing future for ACCESS-ESM1-5 r1i1p1f1
Processing future for ACCESS-ESM1-5 r2i1p1f1
Processing future for ACCESS-ESM1-5 r3i1p1f1
Processing past for AWI-CM-1-1-MR r1i1p1f1...
Processing past for AWI-CM-1-1-MR r2i1p1f1...
Processing past for AWI-CM-1-1-MR r3i1p1f1...
Processing past for AWI-CM-1-1-MR r4i1p1f1...
Processing past for AWI-CM-1-1-MR r5i1p1f1...
Processing future for AWI-CM-1-1-MR r1i1p1f1
Processing past for BCC-CSM2-MR r1i1p1f1...
Processing past for BCC-CSM2-MR r2i1p1f1...
Processing past for BCC-CSM2-MR r3i1p1f1...
Processing future for BCC-CSM2-MR r1i1p1f1
Processing past for CAMS-CSM1-0 r1i1p1f1...
Processing future for CAMS-CSM1-0 r1i1p1f1
Processing future for CAMS-CSM1-0 r2i1p1f1
Processing past for

Calculate indices and output to netcdf

In [13]:
# V_SW index
vpasteddy_em = vpast_em - vpast_em.mean("lon")
vfuteddy_em = vfut_em - vfut_em.mean("lon")
vpasteddy_1mem = vpast_1mem - vpast_1mem.mean("lon")
vfuteddy_1mem = vfut_1mem - vfut_1mem.mean("lon")


vpastsw_em = avg.cosweightlonlat(vpasteddy_em, 250, 265, 25, 37)
vpastsw_em = vpastsw_em.rename("vpastsw_em")
vpastsw_1mem = avg.cosweightlonlat(vpasteddy_1mem, 250, 265, 25, 37)
vpastsw_1mem = vpastsw_1mem.rename("vpastsw_1mem")
vfutsw_em = avg.cosweightlonlat(vfuteddy_em, 250, 265, 25, 37)
vfutsw_em = vfutsw_em.rename("vfutsw_em")
vfutsw_1mem = avg.cosweightlonlat(vfuteddy_1mem, 250, 265, 25, 37)
vfutsw_1mem = vfutsw_1mem.rename("vfutsw_1mem")

# stationary wave ampltiude index
vpast_em_k4to32 = filt.filterk(vpast_em, 4, 32)
vpast_em_k4to32 = vpast_em_k4to32.rename("vpast_em_k4to32")
vpastamp_em = np.sqrt(avg.cosweightlonlat(vpast_em_k4to32**2, 200., 300., 20., 40.))
vpastamp_em = vpastamp_em.rename("vpastamp_em")

vpast_1mem_k4to32 = filt.filterk(vpast_1mem, 4, 32)
vpastamp_1mem = np.sqrt(avg.cosweightlonlat(vpast_1mem_k4to32**2, 200., 300., 20., 40.))
vpastamp_1mem = vpastamp_1mem.rename("vpastamp_1mem")

# Future k>3
vfut_em_k4to32 = filt.filterk(vfut_em, 4, 32)
vfut_em_k4to32 = vfut_em_k4to32.rename("vfut_em_k4to32")

# output vwind indices
vwindindices = xr.merge([vpastsw_em, vfutsw_em, vpastsw_1mem, vfutsw_1mem, vpastamp_em, vpastamp_1mem])
vwindindices.to_netcdf(path=pathout+"cmip6vwindindices.nc")

# output vwind map data
vwindmapdata = xr.merge([vpast_em, vfut_em, vpast_em_k4to32, vfut_em_k4to32])
vwindmapdata.to_netcdf(path=pathout+"cmip6vwindmapdata.nc")


In [8]:
print(vfutsw_1mem-vpastsw_1mem)

<xarray.DataArray (Model: 1)>
array([-4.0330328])
Coordinates:
  * Model    (Model) object 'CanESM5'
